# 1. Download dependencies

In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install --upgrade transformers==4.53.3 datasets[audio] librosa tqdm torchcodec accelerate jiwer

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.9/832.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2

In [2]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)


2025-11-08 08:12:11.034843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762589531.226588      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762589531.291710      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Data Processing

In [3]:
import os
from datasets import load_from_disk, Audio, DatasetDict, disable_caching, enable_caching
# 🔹 1. Load dataset

dataset = load_from_disk("/kaggle/input/dts-pho/vimd_prepared")

Loading dataset from disk:   0%|          | 0/23 [00:00<?, ?it/s]

In [4]:
processor = WhisperProcessor.from_pretrained("vinai/PhoWhisper-small", 
                                             language="vi",      # ✅ đúng
                                             task="transcribe")  # ✅ đúng

model = WhisperForConditionalGeneration.from_pretrained("vinai/PhoWhisper-small")

# ✅ Đây chính là nguyên nhân warning (HG deprecated forced_decoder_ids)
model.generation_config.task = "transcribe"
model.generation_config.language = "vi"
model.config.suppress_tokens = []

new_tokens = [
    "[northern dialect]", "[central dialect]", "[southern dialect]",
    "[male]", "[female]"
]
processor.tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(processor.tokenizer))


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Embedding(50369, 768, padding_idx=50257)

In [5]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    pad_to_multiple_of: Union[int, None] = None

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Tách phần input (audio) và label (text)
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        # Padding audio feature
        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=True,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )

        # Padding text label
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=True,
            return_tensors="pt"
        )

        # Thay padding id trong labels thành -100 (để bỏ qua trong loss)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )
        batch["labels"] = labels.to(torch.long) 

        return batch


# 3. Metric 

In [6]:
import jiwer, numpy as np

def normalize_text(text):
    """Chuẩn hóa text trước khi tính WER"""
    import re
    
    # Lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

def compute_metrics(pred):
    """Tính WER với text normalization"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Decode predictions (skip special tokens và conditioning tokens)
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)

    # Replace -100 trong labels
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Normalize text
    pred_str = [normalize_text(s) for s in pred_str]
    label_str = [normalize_text(s) for s in label_str]

    # Compute WER
    wer = jiwer.wer(label_str, pred_str)
    
    return {"wer": wer}


# 4. Training

In [7]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor, pad_to_multiple_of=8)
training_args = Seq2SeqTrainingArguments(
    # Output & checkpoint
    output_dir="pho-whisper-vimd-conditioned-v2",
    overwrite_output_dir=True,
    save_strategy="steps",
    save_steps=250,  # Save thường xuyên hơn
    save_total_limit=3,  # Giữ 3 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="wer",  # Chọn model tốt nhất dựa trên WER
    greater_is_better=False,
    
    # Training
    max_steps=2000,  # Tăng số steps
    learning_rate=5e-6,  # Giảm learning rate
    warmup_steps=200,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=8,  # Tăng gradient accumulation
    per_device_train_batch_size=2,  # Giảm batch size
    fp16=True,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=250,
    predict_with_generate=True,
    generation_max_length=259,
       
    # GPU & CPU
    dataloader_num_workers=1,  # Tăng workers
    ddp_find_unused_parameters=False,
    dataloader_pin_memory=False,
    
    # Logging
    logging_strategy="steps",
    logging_dir="./logs",
    log_level="info",
    logging_steps=250,
    report_to=["tensorboard"],
    
    # Data management
    remove_unused_columns=False,
    optim="adamw_torch",
    # ✅ GRADIENT CLIPPING
    max_grad_norm=1.0,
)
#model.gradient_checkpointing_enable()


In [8]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],      # ✅ dùng processed_dataset
    eval_dataset=dataset["validation"],  # ✅ dùng processed_dataset
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor.feature_extractor,  # ✅ Cần cho save_pretrained
)

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [9]:

print("🚀 Starting training...")
trainer.train()

***** Running training *****
  Num examples = 11,752
  Num Epochs = 6
  Instantaneous batch size per device = 2
  Training with DataParallel so batch size has been adjusted to: 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 2,000
  Number of trainable parameters = 239,433,984


🚀 Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
250,1.078500,0.262720,0.174321
500,0.187600,0.236986,0.170581
750,0.154400,0.235631,0.155491
1000,0.107300,0.247258,0.169805
1250,0.086100,0.268230,0.165404
1500,0.067900,0.275312,0.153245
1750,0.051100,0.283628,0.160205
2000,0.047100,0.284520,0.160795



***** Running Evaluation *****
  Num examples = 2982
  Batch size = 16
Increase max_length from 259 to 262 since input is conditioned on previous segment.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 25

TrainOutput(global_step=2000, training_loss=0.22250258588790894, metrics={'train_runtime': 36845.2113, 'train_samples_per_second': 1.737, 'train_steps_per_second': 0.054, 'total_flos': 1.84348353724416e+19, 'train_loss': 0.22250258588790894, 'epoch': 5.435670524166099})

# 5. Evaluate

In [10]:
def evaluate_by_dialect(trainer, dataset, split="test"):
    """Đánh giá riêng cho từng dialect - phiên bản không tạo tmp file"""
    
    def filter_dialect(ds, dialect):
        idx = [i for i, row in enumerate(ds) if row["dialect"] == dialect]
        return ds.select(idx)
    
    results = {}

    for dialect in ["northern dialect", "central dialect", "southern dialect"]:
        print(f"\n📊 Evaluating {dialect}...")
        
        subset = filter_dialect(dataset[split], dialect)

        metrics = trainer.evaluate(subset)
        results[dialect] = metrics["eval_wer"]

        print(f"✅ {dialect}: WER = {metrics['eval_wer']:.4f}")

    avg_wer = sum(results.values()) / len(results)
    print(f"\n📈 Average WER: {avg_wer:.4f}")
    
    return results

results = evaluate_by_dialect(trainer, dataset, split="test")

print("\n🎯 FINAL RESULTS")
print(results)


📊 Evaluating northern dialect...



***** Running Evaluation *****
  Num examples = 2688
  Batch size = 16
Increase max_length from 259 to 262 since input is conditioned on previous segment.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous s

✅ northern dialect: WER = 0.0964

📊 Evaluating central dialect...



***** Running Evaluation *****
  Num examples = 705
  Batch size = 16
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Incre

✅ central dialect: WER = 0.4219

📊 Evaluating southern dialect...



***** Running Evaluation *****
  Num examples = 1585
  Batch size = 16
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Increase max_length from 259 to 262 since input is conditioned on previous segment.
Incr

✅ southern dialect: WER = 0.2257

📈 Average WER: 0.2480

🎯 FINAL RESULTS
{'northern dialect': 0.0964238977249849, 'central dialect': 0.42194640881320056, 'southern dialect': 0.2256949343869056}


# 6. Saving Model

In [11]:
print("\n💾 Saving model...")
trainer.save_model("./final_model")
processor.save_pretrained("./final_model")

print("✅ Training completed!")

Saving model checkpoint to ./final_model
Configuration saved in ./final_model/config.json
Configuration saved in ./final_model/generation_config.json



💾 Saving model...


Model weights saved in ./final_model/model.safetensors
Feature extractor saved in ./final_model/preprocessor_config.json
Feature extractor saved in ./final_model/preprocessor_config.json
tokenizer config file saved in ./final_model/tokenizer_config.json
Special tokens file saved in ./final_model/special_tokens_map.json
added tokens file saved in ./final_model/added_tokens.json


✅ Training completed!
